# Frame Extraction

In [2]:
import cv2
import numpy as np

def extract_frames_from_video(video_path, frame_interval=15):
    """
    Extract frames from a single video and return them as a list for later processing.
    
    Args:
    - video_path (str): Path to the input video file.
    - frame_interval (int): Interval for frame extraction (e.g., extract every 30th frame).
    
    Returns:
    - frames (list): A list of frames extracted from the video.
    """
    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Get the total frame count and frame rate
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video.get(cv2.CAP_PROP_FPS)
    
    print(f"Processing {video_path} - Total frames: {total_frames}, FPS: {fps}")

    frames = []
    count = 0

    while True:
        ret, frame = video.read()
        if not ret:
            break

        # Extract the frame every `frame_interval` frames
        if count % frame_interval == 0:
            frames.append(frame)

        count += 1

    # Release the video capture object
    video.release()

    print(f"Extracted {len(frames)} frames from {video_path}")
    
    return frames

# Faces Extraction

In [4]:
import cv2
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN

def extract_highest_confidence_face_from_frames(frames):
    """
    Extract the face with the highest confidence score from a list of frames using MTCNN.
    :param frames: List of frames (images) extracted from the video.
    :return: List of the highest confidence face images from each frame.
    """
    mtcnn = MTCNN()

    face_images = []  

    for idx, frame in enumerate(frames):
        faces = mtcnn.detect_faces(frame)

        if faces:
            highest_confidence_face = None
            max_confidence = 0  # Variable to store the highest confidence

            for face in faces:
                confidence = face['confidence']
                if confidence > max_confidence:
                    max_confidence = confidence
                    highest_confidence_face = face

            if highest_confidence_face:
                x, y, w, h = highest_confidence_face['box']
                face_crop = frame[y:y+h, x:x+w]
                face_images.append(face_crop)

    print(f"Extracted {len(face_images)} faces with the highest confidence.")
    return face_images

# Features Extraction

In [6]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

In [7]:
def preprocess_image(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image_rgb, (224, 224))
    image_preprocessed = preprocess_input(image_resized) 
    image_batch = np.expand_dims(image_preprocessed, axis=0)

    return image_batch    

In [8]:
def feature_extraction(faces):
    features = []

    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')  
    feature_extractor = Model(inputs=model.input, outputs=model.output)  
    
    for face in faces:
        face_n = preprocess_image(face)  
        face_feature = feature_extractor.predict(face_n) 
        features.append(face_feature)  
    
    features = np.concatenate(features, axis=0)   
    features = np.expand_dims(features, axis=0)  
    
    return features

# DeepFakes Detection

In [10]:
def deepfakes_detection(path):
    frames = extract_frames_from_video(path)
    faces = extract_highest_confidence_face_from_frames(frames)
    features =  feature_extraction(faces)
    x = pad_sequences(features, maxlen=108, dtype='float32', padding='post', truncating='post')
    model = load_model("my_model.keras")
    model.compile(optimizer='rmsprop', loss='your_loss_function', metrics=['accuracy'])
    y = model.predict(np.array(x))
    print(y)
    y = (y> 0.5).astype(int)

    if y[0] == 0 : print("Real")
    else: print("Fake")

    return y

In [11]:
y = deepfakes_detection(r"C:\Users\abder\Downloads\I_wish_I_could..._2019.mp4")

Processing C:\Users\abder\Downloads\I_wish_I_could..._2019.mp4 - Total frames: 579, FPS: 29.97002997002997
Extracted 39 frames from C:\Users\abder\Downloads\I_wish_I_could..._2019.mp4
Extracted 39 faces with the highest confidence.
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26

C:\Users\abder\anaconda3\anaconda3-1\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 9 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 594ms/step
[[0.99984455]]
Fake
